In [ ]:
# Version 1.4

# In this version a the velocity was change, the respawn area and now exist a radius of infection
import pygame
import numpy as np
import sys
import random
from pygame.locals import *
import math

# Parameters
WIDTH, HEIGHT = 1800, 1000
NUM_PEOPLE = 50 # This should be 50
PERSON_RADIUS = 5
MAX_SPEED = 2#0.463 
SEPARATION_RADIUS = 20
WALL_MARGIN = 15 # Separación entre las paredes
SPAWN_AREA = (40, 100, 900, 930)  # x_min, x_max, y_min, y_max
GROUP_SIZE = 1                    # Number of people per group
SPAWN_INTERVAL = 3500            # Milliseconds between group spawns
INFECTION_RATE = 0.1  # % de las personas estarán infectadas
ASYMPTOMATIC_RATE = 0.01  # % de las personas serán asintómaticas

# Aleatoriedad de caminos, cantidad de middle path waypoint que una persona puede tener en un recorrido.
CASO_MENOR = 1
CASO_MAYOR = 3

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)  # For path waypoints
BLUE = (0, 0, 255)   # For passable walls
YELLOW = (255, 255, 0)

# Pygame setup
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('Arial', 20)

# Define the initial path waypoints 
'''These are the entries of the space that we are analyzing'''
initial_path_waypoints = [
    np.array([100, 800]),
    np.array([100, 100]),
    ]

# Define the final path waypoints 
'''These are the exits of the space that we are analyzing'''
final_path_waypoints = [
    np.array([350, 600]),
    np.array([350, 650]),
    ]

# Define path waypoints (green)
middle_path_waypoints = [# 975 is the midle point of the market
    np.array([600, 100]),
    np.array([850, 100]),
    np.array([1100, 100]),
    np.array([1350, 100]),
    
    np.array([600, 200]),
    np.array([850, 200]),
    np.array([1100, 200]),
    np.array([1350, 200]),

    np.array([600, 300]),
    np.array([850, 300]),
    np.array([1100, 300]),
    np.array([1350, 300]),

    np.array([600, 400]),
    np.array([850, 400]),
    np.array([1100, 400]),
    np.array([1350, 400]),

    np.array([600, 500]),
    np.array([850, 500]),
    np.array([1100, 500]),
    np.array([1350, 500]),

    np.array([600, 600]),
    np.array([850, 600]),
    np.array([1100, 600]),
    np.array([1350, 600]),
]

checkout_path_waypoints = [
    np.array([275, 100]), 
    
    # Fila 1
    np.array([350, 700]), 
    np.array([350, 750]), 
    np.array([350, 800]), 
    np.array([350, 850]),
    
    np.array([600, 700]), 
    np.array([600, 750]), 
    np.array([600, 800]),
    np.array([600, 850]), 
    
    np.array([850, 700]),
    np.array([850, 750]),
    np.array([850, 800]),
    np.array([850, 850]),
    
    np.array([1100, 700]),
    np.array([1100, 750]),
    np.array([1100, 800]),
    np.array([1100, 850]),
    
    np.array([1350, 700]),
    np.array([1350, 750]),
    np.array([1350, 800]),
    np.array([1350, 850]),
    
    np.array([1600, 700]),
    np.array([1600, 750]),
    np.array([1600, 800]),
    np.array([1600, 850]),
     
]

# Pegamos toda la lista de puntos para dibujar en el mapa las trayectorias 
path_waypoints = initial_path_waypoints + middle_path_waypoints + final_path_waypoints

class Person:
    # Variables iniciales de la clase Person
    def __init__(self, x, y, path, infected=False, asymptomatic = False):
        # Add waiting-related attributes
        self.waiting = False
        self.wait_time = random.uniform(0.5, 2.0)  # Wait between 0.5-2 seconds
        self.wait_start_time = 0

        self.position = np.array([x, y], dtype=float)
        self.velocity = np.random.rand(2) * MAX_SPEED
        self.radius = PERSON_RADIUS
        self.path_index = 0
        self.path_threshold = 20  # Distance to consider waypoint reached
        self.active = True
        self.path = path  # Each person gets its own path
        self.base_color = RED if infected else BLUE
        self.pulse_phase = 0  # Para el efecto de pulso
        
        #self.color = BLUE # Susceptible person
        self.infected = infected
        self.asymptomatic = asymptomatic
        if self.infected:
            self.base_color = RED
        elif self.asymptomatic:
            self.base_color = YELLOW
        else:
            self.base_color = BLUE
            
    def follow_path(self):
        if self.path_index < len(self.path):
            target = self.path[self.path_index]
            direction = target - self.position
            distance = np.linalg.norm(direction)
            
            # Definir los puntos donde NO debe esperar
            x_value = self.path[self.path_index][0]
            is_special_point = x_value in [275, 1625]
            is_first_or_second = self.path_index in [0, 1]
            is_last = self.path_index == len(self.path) - 1

            if distance < self.path_threshold:
                # Solo espera en waypoints intermedios, que NO son especiales
                if (
                    2 < self.path_index < len(self.path) - 1 and  # waypoints intermedios
                    not self.waiting and
                    not is_special_point
                ):
                    # Iniciar espera
                    self.waiting = True
                    self.wait_start_time = pygame.time.get_ticks()
                    self.velocity = np.zeros(2)  # Detenerse
                elif self.waiting:
                    # Checar si terminó de esperar
                    if pygame.time.get_ticks() - self.wait_start_time > self.wait_time * 1000:
                        self.waiting = False
                        self.path_index += 1
                else:
                    # Para los puntos especiales, los dos primeros y el último: avanzar sin esperar
                    self.path_index += 1

                # Si llegó al final, desactiva el agente
                if self.path_index >= len(self.path):
                    self.active = False
            elif distance > 0 and not self.waiting:
                direction_normalized = direction / distance
                self.velocity += direction_normalized * 0.1

    def update(self, people, walls):
        if not self.active:
            return
        
        # Follow the path
        self.follow_path()
        
        if self.waiting:
            return  # Skip movement calculations while waiting
            
        # Rest of the update method remains the same...
        separation = np.zeros(2)
        for other in people:
            if other != self and other.active:
                dist = np.linalg.norm(self.position - other.position)
                if dist < SEPARATION_RADIUS:
                    separation += (self.position - other.position) / (dist + 0.0001)
        
        for wall in walls:
            dist_to_wall = self.distance_to_wall(wall)
            if dist_to_wall < WALL_MARGIN:
                separation += (self.position - wall[:2]) / (dist_to_wall + 0.0001)
        
        self.velocity += separation * 0.1
        speed = np.linalg.norm(self.velocity)
        if speed > MAX_SPEED:
            self.velocity = (self.velocity / speed) * MAX_SPEED
        self.position += self.velocity
        self.position = np.clip(self.position, [0, 0], [WIDTH, HEIGHT])
    
    def distance_to_wall(self, wall):
        x1, y1, x2, y2 = wall
        px, py = self.position
        line_length = np.linalg.norm(np.array([x2 - x1, y2 - y1]))
        if line_length == 0:
            return np.linalg.norm(self.position - np.array([x1, y1]))
        t = max(0, min(1, np.dot([px - x1, py - y1], [x2 - x1, y2 - y1]) / line_length**2))
        closest_point = np.array([x1 + t * (x2 - x1), y1 + t * (y2 - y1)])
        return np.linalg.norm(self.position - closest_point)
    
    def draw(self, screen):
        if not self.active:
            return

        if self.infected or self.asymptomatic:
            # Tiempo animado en segundos
            time = pygame.time.get_ticks() / 250.0  # más suave que *0.005

            # Pulso de tamaño usando seno (entre 2.5x y 3.5x del radio)
            pulse_scale = 3 + 3 * math.sin(time)
            outer_radius = max(1, int(self.radius * pulse_scale))

            # Color pulsante (entre rosa claro y rojo intenso)
            pulse_color = (
                255,
                int(100 + 50 * math.sin(time)),  # valor entre 50 y 150
                int(100 + 50 * math.sin(time))
            )

            # Dibuja círculo externo
            pygame.draw.circle(screen, pulse_color, self.position, outer_radius, width=2)

        # Dibuja la persona (círculo base)
        pygame.draw.circle(screen, self.base_color, self.position, self.radius)

# Define walls (black - solid, must avoid)
walls = [# A vector is define like this [x1, y1, x2, y2]
    # Screen boundaries
    [0, 0, WIDTH, 0],        # Top wall
    [WIDTH, 0, WIDTH, HEIGHT],  # Right wall
    [WIDTH, HEIGHT, 0, HEIGHT],  # Bottom wall
    [0, HEIGHT, 0, 0],        # Left wall
    
    # Interior walls with passages
    [200, 50, 1700, 50], # top wall market
    [1700, 50, 1700, 900], # right wall market
    [1700, 900, 200, 900], # botton wall market
    [200, 900, 200, 150], # left wall market
    
    # Pasillos
    [400, 150, 1500, 150], # division 1
    [400, 250, 1500, 250], # division 2
    [400, 350, 1500, 350], # division 3
    [400, 450, 1500, 450], # division 4
    [400, 550, 1500, 550], # division 5
    #[400, 650, 1500, 650], # division 6
]

# Define passable walls (blue - can walk through)
passable_walls = [
    #[800, 400, 1000, 400],   # Passage between horizontal walls
    #[900, 400, 900, 600]     # Passage between vertical walls
]

# Initialize empty people list
people = []
active_count = 0
# Spawn timer
last_spawn_time = 0
current_group = 0

midle_value = 975 # half of the mall

# Create function to generate path way 
def generate_random_path():
    """Generate a path with random middle waypoints (3-8) between fixed start/end"""
    # Randomly select 3-8 waypoints from the middle path
    num_waypoints = random.randint(CASO_MENOR, CASO_MAYOR)
    selected_waypoints = random.sample(middle_path_waypoints, num_waypoints)
    
    # Combine with fixed start and end points, points of interest and exit
    full_path = selected_waypoints + final_path_waypoints
    
    # Intialize the whole path way
    final_path = initial_path_waypoints
        
    # Make a loop to make the path way smoother for the agente
    i = 0
    for point in full_path:
        # Select each value of the actual objective
        row = point[0]
        column = point[1]
        
        # We calculate the exit of the person in the hallway
        if final_path[-1][0] > midle_value:
            final_path = final_path +  [np.array([1625, final_path[-1][1]])]
        else:
            final_path = final_path + [np.array([275, final_path[-1][1]])]
        
        # We calcule the entry to the hallway
        # The row value is the last value of the list final_path
        final_path = final_path + [np.array([final_path[-1][0], column])]
        
        # We add the actual objective
        final_path = final_path + [point]
        
        # We iterate to next objective        
        i = i + 1
    
    return final_path

# Main loop
running = True
while running:
    # Obtenemos el tiempo actual de la simulación
    current_time = pygame.time.get_ticks()
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            
    # Aparecen nuevos grupos de manera periodica
    if current_time - last_spawn_time > SPAWN_INTERVAL and active_count < NUM_PEOPLE:
        # Generar camino
        group_path = generate_random_path()
        # Generar grupo
        spawn_count = min(GROUP_SIZE, NUM_PEOPLE - active_count)
        for _ in range(spawn_count):
            # Agregamos un efecto random de contagio al grupo
            infected = random.random() < INFECTION_RATE
            asymptomatic = False # Por default asymmptomatic esta en falso
            # Entre un grupo de personas infectadas está la posibilidad de ser asimtomatico
            if infected:
                asymptomatic = random.random() < ASYMPTOMATIC_RATE 
                
            if len(people) < NUM_PEOPLE:
                # Crear nueva persona por grupo
                x = random.randint(SPAWN_AREA[0], SPAWN_AREA[1])
                y = random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                new_person = Person(x, y, group_path, infected, asymptomatic)
                people.append(new_person)
            else:
                # Reactivate existing person
                for person in people:
                    if not person.active:
                        #person.position = np.array([
                        #    random.randint(SPAWN_AREA[0], SPAWN_AREA[1]),
                        #    random.randint(SPAWN_AREA[2], SPAWN_AREA[3])
                        #], dtype=float)
                        #person.velocity = np.random.rand(2) * MAX_SPEED
                        #person.path = group_path
                        #person.path_index = 0
                        person.active = True
                        #person.infected = infected
                        #person.asymptomatic = asymptomatic
                        #person.infected = infected
                        
                        #person.asymptomatic = asymptomatic
                        #if person.infected:
                        #    person.base_color = RED
                        #elif person.asymptomatic:
                        #    person.base_color = YELLOW
                        #else:
                        #    person.base_color = BLUE
                        break
        active_count += spawn_count
        last_spawn_time = current_time
        current_group += 1
    
    screen.fill(WHITE)
    
    # Draw path waypoints
    for i, point in enumerate(path_waypoints):
        pygame.draw.circle(screen, GREEN, point.astype(int), 10)
        if i > 0:
            pygame.draw.line(screen, (200, 200, 200), 
                           path_waypoints[i-1], point, 2)
            
    for i, point in enumerate(checkout_path_waypoints):
        pygame.draw.circle(screen, YELLOW, point.astype(int), 10)
    
    # Draw walls (solid)
    for wall in walls:
        pygame.draw.line(screen, BLACK, (wall[0], wall[1]), (wall[2], wall[3]), 5)
    
    # Draw passable walls (can walk through)
    for wall in passable_walls:
        pygame.draw.line(screen, RED, (wall[0], wall[1]), (wall[2], wall[3]), 3)
    
    # Update and draw people
    active_count = 0
    for person in people:
        person.update(people, walls)
        person.draw(screen)
        if person.active:
            active_count += 1
            
    
    # Display spawn info
    info_text = f"Active: {active_count}/{NUM_PEOPLE}  Next spawn in: {max(0, (SPAWN_INTERVAL - (current_time - last_spawn_time))//1000) }s"
    text_surface = font.render(info_text, True, BLACK)
    screen.blit(text_surface, (10, 10))
    
    pygame.display.flip()
    clock.tick(60)
    
pygame.quit()
sys.exit()

SystemExit: 

In [5]:
import numpy as np
final_path_waypoints = [
    np.array([300, 100]),
    np.array([50, 100]),
    np.array([450, 1004]),
]
print(final_path_waypoints[-1][0])

450
